In [6]:
## Import Packages
from __future__ import print_function

import numpy as np

#Plotting Packages
import matplotlib as mpl
import matplotlib.cm as cmplt
import matplotlib.pyplot as plt
from matplotlib import rcParams
from tqdm import tqdm

#I/O
import glob
import copy
import time
import os
import scipy.io as sio

#Plotting Style
%matplotlib inline
plt.style.use('dark_background')
rcParams['text.usetex'] = True
rcParams['axes.titlesize'] = 20
rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16
rcParams['legend.fontsize'] = 12
rcParams['axes.labelsize'] = 20
rcParams['font.family'] = 'sans-serif'

In [7]:
import aotools
from aotools.turbulence import infinitephasescreen
import imageio
from pytransit import QuadraticModel

In [8]:
tm = QuadraticModel()
times = np.linspace(-0.3,0.3,101)
tm.set_data(times)

In [9]:
transit_curve = tm.evaluate(k=0.1, ldc=[0.2, 0.1], t0=0.0, p=1.0, a=1.3, i=0.5*np.pi)

In [10]:
def star_flux(flux,sigma_x = 1.,sigma_y = 2.,size = 256):
    x = np.linspace(-size/2, size/2-1, size)
    y = np.linspace(-size/2, size/2-1, size)
    x, y = np.meshgrid(x, y)
    
    z = flux*(1/(2*np.pi*sigma_x*sigma_y) * np.exp(-(x**2/(2*sigma_x**2)
         + y**2/(2*sigma_y**2))))
    return z

def convolver(turb,im2):
    if1 = turb
    if2 = np.fft.fftshift(np.fft.fft2(im2))
    conv = np.multiply(if1,if2)
    iif = np.fft.ifft2(conv)
    return iif

def noise(img,gauss_sig=5):
    #convert from 0-1 norm to unit16
    in_img = 32767*img
    out_img = np.random.poisson(in_img)
    out_img += np.random.normal(0, gauss_sig, (img.shape[0],img.shape[1])).astype(np.int64)
    return np.abs(out_img)

In [11]:
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    if pad_width[1] != 0:                      # <-- the only change (0 indicates no padding)
        vector[-pad_width[1]:] = pad_value

In [13]:
for indx, i in enumerate(tqdm(transit_curve)):
    z = star_flux(i)
    out_z = 32767*z
    imageio.imwrite('data1/transit_test_gt_{}.tiff'.format(indx), out_z)
    for j in range(40):
        scrn = infinitephasescreen.PhaseScreenKolmogorov(256, 0.4/256, 0.05, 0.5, random_seed=107*indx)
        #scrn = np.pad(scrn, ((0,0),(10,10)), pad_with)
        #final_img = noise(np.abs(convolver(scrn.scrn,z)))
        final_img = noise(np.pad((np.abs(convolver(scrn.scrn,z))), ((0,0),(10,10)), pad_with))
        imageio.imwrite('data1/transit_test_{}_slice_{}.tiff'.format(indx,j), final_img)

100%|██████████████████████████████████████| 101/101 [1:14:54<00:00, 44.50s/it]
